## Example: Working with mesh models

The main feature of HydroMT is to facilitate the process of building and analyzing spatial geoscientific models with a focus on water system models. It does so by automating the workflow to go from raw data to a complete model instance which is ready to run and to analyse model results once the simulation has finished. 

This notebook will explore how to work with mesh models.

In [ ]:
# import hydromt and setup logging
import hydromt
from hydromt.utils import setuplog

# other imports
import matplotlib.pyplot as plt
import geopandas as gpd

logger = setuplog("working with mesh models", log_level=10)

### Check if mesh_model is available

To know which models are available within your active environment, you can use global `MODELS` variable in hydromt

In [ ]:
# generic model classes
print(f"Generic model classes: {hydromt.MODELS.generic}")
# model classes from external plugin
print(f"Model classes from plugins: {hydromt.MODELS.plugins}")

Here you may only see the generic models ``grid_model``, ``vector_model`` and ``network_model``. There is one more generic model within HydroMT ``mesh_model`` which is only available if the additional python mesh dependency *xugrid* is available in the activated environment. If not add xugrid to your environment.

### Mesh component properties and preparation with setup_mesh2d method

The main component of ``MeshModel`` is the ``mesh`` object. The ``mesh`` object in HydroMT is defined using the [UgridDataset](https://deltares.github.io/xugrid/api/xugrid.UgridDataset.html#xugrid.UgridDataset) object of the [xugrid](https://deltares.github.io/xugrid/).

The mesh object is able to contain 1D, 2D and/or 3D unstructured grid(s) according to the UGRID conventions. Several grids are supported and each grid should have a name to be able to select or prepare data for a specific one.

Let's have a look at an example of ``MeshModel.mesh`` and the available properties:

In [ ]:
# Instantiate a mesh model
mod = hydromt.MeshModel(
    root="data/mesh_model",
    mode="r",
)
# Read the mesh
mod.read_mesh(fn='mesh1d2d.nc', crs=3857)
# Print the mesh
print(type(mod.mesh))
mod.mesh

In HydroMT, you have a couple of [properties](https://deltares.github.io/hydromt/latest/api.html#id8) that allow you to get the names of the different grids in mesh *mesh_names*, extract a specific grid from the mesh with *mesh_datasets* or without data *mesh_grids* or convert the grids to geopandas.GeoDataFrame *mesh_gdf*.

Let's check the available grids in our mesh and plot them:

In [ ]:
mod.mesh_names

In [ ]:
# Plot the different mesh grids
fig, ax = plt.subplots(figsize = (5, 6))
# Mesh2d
mod.mesh_grids["mesh2d"].plot(ax=ax, facecolor="none", edgecolor="k", label="mesh2d")
# Mesh1d
mod.mesh_grids["network1d"].plot(ax=ax, label="network1d")

# Plot legends
ax.legend()
ax.set_title("Mesh grids")
ax.set_xlabel("x")
ax.set_ylabel("y")


With HydroMT, you can create a 2D mesh grid using the ``setup_mesh2d`` method. This method parses the [HydroMT region option](https://deltares.github.io/hydromt/latest/user_guide/model_region.html) to define the geographic region of interest and mesh of the MeshModel to build and once done, adds **region** into the `geoms` component and 2D grid into the `mesh` component. You can check the required arguments in the [docs](https://deltares.github.io/hydromt/latest/_generated/hydromt.MeshModel.setup_mesh2d.html).

With this method, you can generate a 2D regular mesh within a bounding box or geometry, or from reading an existing mesh file. We will see an example for both of them.

In [ ]:
# First instantiate a new mesh model
root = "tmp_mesh_model"
mod = hydromt.MeshModel(
    root=root,
    mode="w",
    data_libs=["artifact_data=v1.0.0", "data/vito_reclass.yml"],
    logger=logger,
)

In [ ]:
# Generate a 2D mesh from file
region = {'mesh': 'data/mesh_model/mesh1d2d.nc', 'grid_name': 'mesh2d'}
mod._mesh = None # delete the previous mesh
mod.setup_mesh2d(
    region=region,
    crs = 3857,
    grid_name="mesh2d",
)
# Save and reset mesh
mesh1 = mod.mesh_gdf["mesh2d"].copy()
mod._mesh = None

# Generate a 2D mesh from bounding box and resolution
region = {'bbox': [12.09, 46.45, 12.20, 46.50]}
mod.setup_mesh2d(
    region=region,
    res=1000,
    crs=3857,
    grid_name="mesh2d",
)
# Save and reset mesh
mesh2 = mod.mesh_gdf["mesh2d"].copy()



In [ ]:
# Plot
fig = plt.figure(figsize=(10, 4))
ax = plt.subplot()
# plot mesh1 mesh2 gdf without facecolor
mesh1.plot(ax=ax, facecolor="none", edgecolor="k", label="mesh from bbox")
mesh2.plot(ax=ax, facecolor="pink", edgecolor="r", label="mesh from file")

### Add data to 2D mesh

To prepare and resample data to a 2D grid in mesh, the core for now has two methods available: [setup_mesh2d_from_rasterdataset](https://deltares.github.io/hydromt/latest/_generated/hydromt.MeshModel.setup_mesh2d_from_rasterdataset.html#hydromt.MeshModel.setup_mesh2d_from_rasterdataset) and [setup_mesh2d_from_raster_reclass](https://deltares.github.io/hydromt/latest/_generated/hydromt.MeshModel.setup_mesh2d_from_raster_reclass.html#hydromt.MeshModel.setup_mesh2d_from_raster_reclass).

But back to our example, let's add both a DEM map from the data source *merit_hydro_1k* and a landuse and manning roughness map based on reclassified landuse data from the *vito* dataset to the mesh2d grid of our model mesh object.

For this we will use xugrid Regridder methods to interpolate raster data to our mesh. Our current model resolution is 500*500 meters so for the different data source we will resample using the methods:

- merit_hydro_1k: *centroid* as the resolution of the data is 1000*1000 meters. You could also choose mean or geometric_mean or others if you wish.
- landuse: *mode* (most representative) as we want to conserve the land use classification values.
- manning: the resolution of vito is 250*250 meters so we can choose to use for example the *mean* method.

In [ ]:
mod.setup_mesh2d_from_rasterdataset(
    raster_fn="merit_hydro_1k",
    grid_name="mesh2d",
    variables="elevtn",
    fill_method=None,
    resampling_method="centroid",
    rename={"elevtn": "DEM"},
)
mod.setup_mesh2d_from_raster_reclass(
    raster_fn="vito",
    grid_name="mesh2d",
    fill_method="nearest",
    reclass_table_fn="vito_reclass",  # Note: from local data catalog
    reclass_variables=["landuse", "manning"],
    resampling_method=["mode", "mean"],
)

In [ ]:
# Plot
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 4), sharey=True)
mod.mesh["DEM"].ugrid.plot(ax=axes[0], cmap="terrain")
axes[0].set_title("Elevation")

mod.mesh["landuse"].ugrid.plot(ax=axes[1], cmap="Pastel1")
axes[1].set_title("Landuse")

mod.mesh["manning"].ugrid.plot(ax=axes[2], cmap="viridis")
axes[2].set_title("Manning roughness")